In [8]:
import json
from typing import List, Tuple

import fsspec
import gpxpy
import gpxpy.gpx

In [9]:
def load_route_json_from_gcs(bucket: str, path: str, token: str = "google_default") -> dict:
    """Load a JSON file from GCS as a Python dict using fsspec."""
    uri = f"gs://{bucket}/{path}" # construct GCS URI
    fs = fsspec.filesystem("gcs", token=token) # connect to GCS
    with fs.open(uri, "rb") as f:
        return json.load(f)


In [10]:
def extract_coords(route_json: dict) -> Tuple[List[Tuple[float, float]], List[float]]:
    """Extract (lat, lon) pairs and elevation data from a GeoJSON/ORS-like route dict.

    Returns:
        Tuple of (coords, elevations) where coords is list of (lat, lon) and elevations is list of heights
    """
    # Typical ORS/GeoJSON: FeatureCollection -> first feature -> geometry
    if route_json.get("type") == "FeatureCollection":
        features = route_json.get("features", [])
        if not features:
            raise ValueError("No features found in route JSON")
        geom = features[0].get("geometry", {})
    else:
        # Fallback: assume the dict itself has geometry
        geom = route_json.get("geometry", {})

    gtype = geom.get("type")
    coords = geom.get("coordinates", [])

    coord_list = []
    elev_list = []

    if gtype == "LineString":
        # coords is list of [lon, lat, elev] triplets
        for lon, lat, elev in coords:
            coord_list.append((lat, lon))
            elev_list.append(elev)
    elif gtype == "MultiLineString":
        for line in coords:
            for lon, lat, elev in line:
                coord_list.append((lat, lon))
                elev_list.append(elev)
    else:
        raise ValueError(f"Unsupported geometry type: {gtype}")

    return coord_list, elev_list

In [11]:
def create_gpx_from_coords(
    coords: List[Tuple[float, float]],
    elevations: List[float],
    route_name: str = "Cycling Route",
    route_description: str = "Generated by Cycle More App"
) -> gpxpy.gpx.GPX:
    """Create a GPX object from coordinate and elevation data.

    Args:
        coords: List of (lat, lon) tuples
        elevations: List of elevation values in meters (must match coords length)
        route_name: Name for the route (default: "Cycling Route")
        route_description: Description for the route

    Returns:
        gpxpy.gpx.GPX object ready to be saved

    Raises:
        ValueError: If coords and elevations lists have different lengths
    """
    if len(coords) != len(elevations):
        raise ValueError(f"Coords ({len(coords)}) and elevations ({len(elevations)}) must have same length")

    # Create GPX object
    gpx = gpxpy.gpx.GPX()

    # Add metadata
    gpx.name = route_name
    gpx.description = route_description

    # Create a track
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx_track.name = route_name
    gpx.tracks.append(gpx_track)

    # Create a segment in the track
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)

    # Add all points to the segment
    for (lat, lon), elev in zip(coords, elevations):
        gpx_segment.points.append(
            gpxpy.gpx.GPXTrackPoint(
                latitude=lat,
                longitude=lon,
                elevation=elev
            )
        )

    return gpx


def save_gpx_to_file(gpx: gpxpy.gpx.GPX, output_path: str) -> None:
    """Save a GPX object to a file.

    Args:
        gpx: GPX object to save
        output_path: Path where the GPX file should be saved
    """
    with open(output_path, 'w') as f:
        f.write(gpx.to_xml())
    print(f"✓ GPX file saved to: {output_path}")


def coords_to_gpx_file(
    coords: List[Tuple[float, float]],
    elevations: List[float],
    output_path: str,
    route_name: str = "Cycling Route",
    route_description: str = "Generated by Cycle More App"
) -> None:
    """One-step function: Convert coords + elevations directly to GPX file.

    Args:
        coords: List of (lat, lon) tuples
        elevations: List of elevation values in meters
        output_path: Path where the GPX file should be saved
        route_name: Name for the route
        route_description: Description for the route
    """
    gpx = create_gpx_from_coords(coords, elevations, route_name, route_description)
    save_gpx_to_file(gpx, output_path)

## Complete Workflow: GeoJSON → GPX

This example shows the complete workflow from loading a route from GCS to exporting it as a GPX file.

In [12]:
# Example 1: Complete workflow from GCS to GPX file
bucket = "cycle_more_bucket"
path = "raw_ors_data/belgium/Amsterdam_route_342962.json"

# Step 1: Load the route JSON from GCS
route_json = load_route_json_from_gcs(bucket, path)

# Step 2: Extract coordinates and elevations
coords, elevations = extract_coords(route_json)
print(f"Extracted {len(coords)} points with elevation data")

# Step 3: Generate GPX file
output_path = "amsterdam_route_342962.gpx"
coords_to_gpx_file(
    coords=coords,
    elevations=elevations,
    output_path=output_path,
    route_name="Amsterdam Cycling Route 342962",
    route_description="Scenic cycling route through Amsterdam"
)

print(f"\n✓ GPX export complete! File ready for GPS devices and apps like Strava, Komoot, etc.")

Extracted 328 points with elevation data
✓ GPX file saved to: amsterdam_route_342962.gpx

✓ GPX export complete! File ready for GPS devices and apps like Strava, Komoot, etc.


In [7]:
# Example 2: Two-step approach (useful if you want to inspect or modify GPX before saving)

# Create GPX object
gpx = create_gpx_from_coords(
    coords=coords,
    elevations=elevations,
    route_name="Amsterdam Route",
    route_description="Custom cycling route"
)

# Inspect GPX statistics
print(f"Route name: {gpx.name}")
print(f"Total points: {len(gpx.tracks[0].segments[0].points)}")
print(f"GPX XML preview (first 500 chars):\n{gpx.to_xml()[:500]}...")

# Save to file
save_gpx_to_file(gpx, "amsterdam_route_custom.gpx")

Route name: Amsterdam Route
Total points: 328
GPX XML preview (first 500 chars):
<?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="gpx.py -- https://github.com/tkrajina/gpxpy">
  <metadata>
    <name>Amsterdam Route</name>
    <desc>Custom cycling route</desc>
  </metadata>
  <trk>
    <name>Amsterdam Route</name>
    <trkseg>
      <trkpt lat="52.380694" lon="4.898811...
✓ GPX file saved to: amsterdam_route_custom.gpx


### Integration with find_route_path

If you have the `find_route_path` function from Visualisation_route notebook, you can easily export routes by ID:

In [ ]:
# If you have find_route_path from Visualisation_route notebook:
# from google.cloud import storage

# def find_route_path(bucket_name: str, route_id: int, prefix: str = "all_routes/"):
#     client = storage.Client()
#     bucket = client.bucket(bucket_name)
#     search_pattern = f"route_{route_id}.json"
#     matches = [
#         blob.name
#         for blob in client.list_blobs(bucket, prefix=prefix)
#         if blob.name.endswith(search_pattern)
#     ]
#     if not matches:
#         raise FileNotFoundError(f"No file ending with {search_pattern} found in {prefix}")
#     return matches[0]

# # Then export by route ID:
# route_id = 342962
# path = find_route_path("cycle_more_bucket", route_id)
# route_json = load_route_json_from_gcs("cycle_more_bucket", path)
# coords, elevations = extract_coords(route_json)
# coords_to_gpx_file(coords, elevations, f"route_{route_id}.gpx", route_name=f"Route {route_id}")

### Batch Export for Multiple Routes

Export multiple routes at once - perfect for recommendation systems where users get 5 similar route options.

In [13]:
import os
from google.cloud import storage


def find_route_path(bucket_name: str, route_id: int, prefix: str = "all_routes/") -> str:
    """Find the GCS path for a route by its ID.

    Args:
        bucket_name: Name of the GCS bucket
        route_id: Route ID to search for
        prefix: GCS prefix to search in

    Returns:
        Full path to the route JSON file in GCS
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    search_pattern = f"route_{route_id}.json"

    matches = [
        blob.name
        for blob in client.list_blobs(bucket, prefix=prefix)
        if blob.name.endswith(search_pattern)
    ]

    if not matches:
        raise FileNotFoundError(f"No file ending with {search_pattern} found in {prefix}")

    if len(matches) > 1:
        print(f"⚠️ Warning: multiple matches found for route {route_id}. Using the first one.")
        print(matches)

    return matches[0]


def export_multiple_routes_to_gpx(
    route_ids: List[int],
    bucket: str,
    output_dir: str = "exported_routes",
    prefix: str = "all_routes/"
) -> List[str]:
    """Export multiple routes by ID to GPX files.

    Perfect for recommendation systems where users get 5 similar route options.

    Args:
        route_ids: List of route IDs to export
        bucket: GCS bucket name
        output_dir: Directory to save GPX files (created if doesn't exist)
        prefix: GCS prefix where routes are stored

    Returns:
        List of output file paths

    Example:
        # Export 5 recommended routes
        recommended_ids = [342962, 123456, 789012, 456789, 999888]
        files = export_multiple_routes_to_gpx(recommended_ids, "cycle_more_bucket")
        print(f"Exported {len(files)} routes for user to download")
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    gpx_files = []
    successful = 0
    failed = []

    print(f"Exporting {len(route_ids)} routes to GPX...")
    print("-" * 50)

    for i, route_id in enumerate(route_ids, 1):
        try:
            # Find and load route
            path = find_route_path(bucket, route_id, prefix)
            route_json = load_route_json_from_gcs(bucket, path)
            coords, elevations = extract_coords(route_json)

            # Export with ID in filename
            output_path = os.path.join(output_dir, f"route_{route_id}.gpx")
            coords_to_gpx_file(
                coords=coords,
                elevations=elevations,
                output_path=output_path,
                route_name=f"Cycle More Route {route_id}",
                route_description=f"Recommended cycling route (ID: {route_id})"
            )

            gpx_files.append(output_path)
            successful += 1
            print(f"[{i}/{len(route_ids)}] ✓ Route {route_id}")

        except Exception as e:
            failed.append((route_id, str(e)))
            print(f"[{i}/{len(route_ids)}] ✗ Route {route_id}: {e}")

    print("-" * 50)
    print(f"\n✓ Successfully exported: {successful}/{len(route_ids)} routes")

    if failed:
        print(f"✗ Failed: {len(failed)} routes")
        for route_id, error in failed:
            print(f"  - Route {route_id}: {error}")

    return gpx_files

In [14]:
# Example: Export 5 recommended routes for a user
# This simulates your recommendation system returning 5 similar routes

recommended_route_ids = [342962, 342963, 342964, 342965, 342966]  # Replace with actual IDs from your recommendation system
recommended_route_ids = [10058, 10113048, 10139557, 10188937, 10222038]  # Replace with actual IDs from your recommendation system

gpx_files = export_multiple_routes_to_gpx(
    route_ids=recommended_route_ids,
    bucket="cycle_more_bucket",
    output_dir="recommended_routes"  # Creates this folder if it doesn't exist
)

print(f"\n📁 All GPX files saved in: recommended_routes/")
print(f"📤 Users can now download these {len(gpx_files)} files and upload to Strava/Komoot!")
print("\nGenerated files:")
for file in gpx_files:
    print(f"  - {file}")

Exporting 5 routes to GPX...
--------------------------------------------------
✓ GPX file saved to: recommended_routes/route_10058.gpx
[1/5] ✓ Route 10058
✓ GPX file saved to: recommended_routes/route_10113048.gpx
[2/5] ✓ Route 10113048
✓ GPX file saved to: recommended_routes/route_10139557.gpx
[3/5] ✓ Route 10139557
⚠️ Warning: multiple matches found for route 10188937. Using the first one.
['all_routes/belgium_route_10188937.json', 'all_routes/netherlands_route_10188937.json']
✓ GPX file saved to: recommended_routes/route_10188937.gpx
[4/5] ✓ Route 10188937
✓ GPX file saved to: recommended_routes/route_10222038.gpx
[5/5] ✓ Route 10222038
--------------------------------------------------

✓ Successfully exported: 5/5 routes

📁 All GPX files saved in: recommended_routes/
📤 Users can now download these 5 files and upload to Strava/Komoot!

Generated files:
  - recommended_routes/route_10058.gpx
  - recommended_routes/route_10113048.gpx
  - recommended_routes/route_10139557.gpx
  - rec